In [121]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [122]:
gmsh.open("buckling-3.geo")

Info    : Reading 'buckling-3.geo'...
Info    : Meshing 1D...                                                                                                     
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Line)
Info    : [ 50%] Meshing curve 9 (Line)
Info    : [ 50%] Meshing curve 10 (Line)
Info    : [ 60%] Meshing curve 11 (Line)
Info    : [ 60%] Meshing curve 12 (Line)
Info    : [ 70%] Meshing curve 13 (Line)
Info    : [ 70%] Meshing curve 14 (Line)
Info    : [ 80%] Meshing curve 15 (Line)
Info    : [ 80%] Meshing curve 16 (Line)
Info    : [ 90%] Meshing curve 17 (Line)
Info    : [ 90%] Meshing curve 18 (Line)
Info    : [100%] Meshing curve 19 (Line)
Info    : [100%] Meshing curve 20 (Line)
Info    : Done meshing 1D 

In [123]:
mat = FEM.material("body", E=2.0e5, ν=0.3)
problem = FEM.Problem([mat], type="Solid", bandwidth="none")

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 61)


LowLevelFEM.Problem("buckling-3", "Solid", 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 775)

In [124]:
l = gmsh.parser.getNumber("l")[1]
a = gmsh.parser.getNumber("a")[1]
F = -43000

-43000

In [125]:
supp = FEM.displacementConstraint("supp", ux=0, uy=0, uz=0)
supp0 = FEM.displacementConstraint("left", ux=0, uy=0, uz=0)
supp1 = FEM.displacementConstraint("load", uy=0, uz=0)
supp2 = FEM.displacementConstraint("load", ux=0, uy=0, uz=0)
load = FEM.load("load", fx=F / a^2)
load0 = FEM.load("right", fx=F / a^2)

("right", -430.0, 0, 0)

In [126]:
B = FEM.solveBuckling(problem, [load0], [supp0, supp1])

LowLevelFEM.Eigen([7.741311289617717, 7.773445219045516, 20.657313048003566, 20.70147261198755, 36.009684847852036, 36.079386367813605], [3.835414739076605e-18 1.9598270593912757e-18 … -6.578881151772356e-18 6.190545322957365e-18; 1.2448453330002649e-17 6.248865906871235e-18 … -1.8509699188271046e-17 1.1011818400420892e-17; … ; -2.195143462634065e-5 0.007361238544376518 … -0.000254337932698498 0.014852022135723144; 0.007368036186681596 -2.209502024277976e-5 … 0.015026326210510747 -0.00025130540719047156])

In [127]:
B.f

6-element Vector{Float64}:
  7.741311289617717
  7.773445219045516
 20.657313048003566
 20.70147261198755
 36.009684847852036
 36.079386367813605

In [128]:
bk = FEM.showBucklingResults(problem, B, visible=false)

0

In [129]:
M = FEM.solveModalAnalysis(problem, load=[load0], bc=[supp0], n=10, fₘᵢₙ=1.01)

LowLevelFEM.Eigen([69.57469975960547, 69.57469977788847, 4553.43561899433, 4553.435619122388, 6942.933038176677, 12658.175747332913, 12783.453898848917, 12783.453929208134, 20811.49854416199, 23510.701574610528], [7.993178908073524e-20 -1.2207961391325385e-19 … 8.496414120480259e-18 -1.790744106612404e-17; -4.3767490832749716e-20 4.512596211698983e-20 … 7.536302162209485e-20 -3.1265101566040136e-18; … ; 0.06820818190426926 -0.033611405913367226 … 0.025085974403650367 -0.03862755581684184; 0.015511942275861815 0.061342606915151254 … -0.025085974587650075 -0.0019448444299811588])

In [130]:
ma = FEM.showModalResults(problem, M, visible=true)

1

In [131]:
gmsh.view.option.setNumber(bk, "DisplacementFactor", 300)
gmsh.view.option.setNumber(ma, "DisplacementFactor", 300)

In [132]:
E = problem.material[1].E
A = a * a
I = a^4 / 12
i = √(I / A)
β = 0.707
l0 = β * l
λ = l0 / i
display("λ = $λ")
σₖᵣ = π^2 * E / λ^2
Fₖᵣ = σₖᵣ * A
display("theory: Fₖᵣ = $Fₖᵣ")

Fkrfem = -F * B.f[1]
display("fem: Fₖᵣ = $Fkrfem")

"λ = 24.491198419023924"

"theory: Fₖᵣ = 329086.1974012605"

"fem: Fₖᵣ = 332876.38545356184"

In [133]:
n = 1
ρ = problem.material[1].ρ
ω0 = π / 2 / l * √(E / ρ) / 2π
ω0i = n^2 * π^2 / l^2 * √(E * I / (ρ * A)) / 2π
ωi = n^2 * π^2 / l^2 * √(E * I / (ρ * A)) * √(1 + l^2 * 0F / (n^2 * π^2 * E * I)) / 2π
display("ω0 = $ω0")
display("i = $n")
display("ω0i = $ω0i")
display("ωi = $ωi")
display(M.f)

"ω0 = 12618.86162812672"

"i = 1"

"ω0i = 2288.808319845581"

"ωi = 2288.808319845581"

10-element Vector{Float64}:
    69.57469975960547
    69.57469977788847
  4553.43561899433
  4553.435619122388
  6942.933038176677
 12658.175747332913
 12783.453898848917
 12783.453929208134
 20811.49854416199
 23510.701574610528

In [134]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [135]:
gmsh.finalize()